### SQL Homework
Use this notebook to answer the questions.
It can be in the same project as the previous homework.  
When you are ready, **upload** to your github repo, and send me the link (just zip a txt file with the repo's address, and upload it as the homework).  
If your repo is private, invite me: balazs.balogh@cubixedu.com.

#### Import the SparkSession, create it then load the taxi data (yellow_tripdata_2024-08.parquet)

#### 1. What is the total fare amount for all trips?  
Please round the answer to two decimal places.

#### 2. Show the maximum fare amount, minimum fare amount, and average fare amount for each payment type. Order by payment type.
Round where you need to two decimal places.

#### 3. For trips with a fare amount greater than 20, what is the total tip amount for each day (based on the tpep_pickup_datetime)?
Round the tip to two decimal places, and order the results from highest total tip amount.  
Hint: Check DATE() function, to convert tpep_pickup_datetime to date, to get only the YYYY-MM-DD.

#### 4. For each trip, show the fare amount along with a column that indicates if the trip was "expensive" (greater than 30) or "cheap" (less than or equal to 30).
Hint: Use CASE WHEN for deciding on expensive, or cheap.

#### 5. Find the first trip (based on tpep_pickup_datetime) for each VendorID and display the fare amount.
Hint: You can use CTE with ROW_NUMBER().

#### 7. Calculate the average trip distance for each VendorID, and assign a label of 'Above Average' or 'Below Average' for each trip based on the distance relative to the VendorID’s average trip distance.
Hint: CTE joined back to the main DataFrame.